In [1]:
!pip install nltk
!pip install spacy
!python -m spacy download fr_core_news_sm
!pip install scikit-learn
!pip install imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 89.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 118.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 131.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 58.8 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [2]:
import spacy  
import pandas as pd  
import numpy as np
from collections import Counter
import math
import re  
import nltk 
from nltk.corpus import stopwords  
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

In [3]:
#import du fichier 

file_path = os.path.join(os.getcwd(), "20220913_offers.csv")
df = pd.read_csv(file_path, sep=";", low_memory=False)

columns = ["job_title", "contractType", "description_job","description_profil", "description_entreprise","entreprise_nom"]
df = df[columns]
df['offre_emploi'] = df[['description_job', 'description_profil', 'description_entreprise']].apply(
    lambda x: ' '.join(x.dropna()), axis=1
)

print(df.head())

                     job_title contractType  \
0               Cordiste (H/F)          MIS   
1  Employé libre service (H/F)          MIS   
2   Tuyauteur industriel (H/F)   Alternance   
3         Data scientist (H/F)        Stage   
4             Chirurgien (H/F)          CDI   

                                     description_job  \
0  Votre agence R INTERIM Tarbes recherche pour l...   
1  R intérim 41 Rue Jules Ferry à Saint-Brieuc In...   
2  R Interim recherche un tuyauteur pour l'un de ...   
3  Rejoignez un collectif animé par le goût du dé...   
4  Emploi Chirurgien Gynécologue Coutances 50 : U...   

                                  description_profil  \
0  Vous êtes titulaire d'un CQP ou d'un diplôme d...   
1  Vous avez le sens du service et vous vous adap...   
2  Fort d'une première expérience ou stage en tuy...   
3                                                NaN   
4  Chirurgien Gynécologue - inscrit/inscriptible ...   

                              description_e

In [4]:
df["offre_emploi"] = df["offre_emploi"].str.lower()

nltk.download('stopwords')  
stop_words = set(stopwords.words('french'))

nlp = spacy.load("fr_core_news_sm")

# Fonction de nettoyage avec lemmatisation
def clean_and_lemmatize(text):
    if isinstance(text, str):  
        text = text.lower()  # Minuscule
        text = re.sub(r'\d+', '', text)  # Suppression des chiffres
        text = re.sub(r'[^\w\s]', '', text)  # Suppression de la ponctuation
        doc = nlp(text)  # Analyse du texte avec SpaCy
        words = [token.lemma_ for token in doc if token.text not in stop_words]  # Lemmatisation + suppression stopwords
        return " ".join(words)  # Reconstituer le texte nettoyé
    return ""

df["offre_emploi"] = df["offre_emploi"].apply(clean_and_lemmatize)

[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


KeyboardInterrupt: 

In [ ]:
#première classification : on regarde le nombre d'occurrences de chaque mot associé à chaque 
#catégorie, à partir de là on établit un score pour chaque catégorie et on assigne à une offre 
#d'emploi la catégorie pour laquelle le score est le plus élevé 

def classify_offer(text, keywords_dict):
    scores = {category: 0 for category in keywords_dict}
    
    for category, keywords in keywords_dict.items():
        scores[category] += sum(text.count(keyword) for keyword in keywords)
    
    return max(scores, key=scores.get) if max(scores.values()) > 0 else "inconnu"

#dictionnaire avec mots clés :

keywords_dict = {
    "Excellence, Leadership et Performance": [
        "leader", "leadership", "lead", "diriger", "décider", "excellence", "exceller", "haut niveau", "prestigieux", "prestige", "réputation", "réputé", "certification",
        "expert", "expertise", "compétence", "qualifié", "qualité", "performance", "performant", "responsabilité"
    ],
    "Accompagnement et Evolution Professionnelle": [
        "accompagnement", "accompagner", "évolution", "évoluer", "formation", "formation continu", "développement", "développer", "carrière", "professionnel", "promotion",
        "succès", "opportunité", "soutien"
    ],
    "Proximité et Ancrage Local": [
        "proximité", "ancrage local", "lien", "relation client", "territoire", "local", "régional",
        "implantation", "département", "acteur", "région"
    ],
    "Relations Humaines et Collectif": [
        "esprit déquipe", "équipe", "familial", "convivialité", "convivial", "entraide", "partage", "solidarité", "respect",
        "famille", "cohésion"
    ],
    "Engagement Social et Bien-être": [
        "rse", "social", "citoyen", "planete", "durable", "développement durable", "inclusion", "handicap", "diversité", "impact social",
        "bienêtre", "environnement", "santé"
    ]#,
    #"Innovation et Technologie": [
    #    "innovation", "tech", "technologie", "transformation digital", "croissance", "expansion", "startup", "modernisation", "intelligence artificiel", "ia", "ai", "artificiel", "logiciel",
    #    "inovant", "r&d"
    #]
}

df["valeur_dominante"] = df["offre_emploi"].apply(lambda x: classify_offer(x, keywords_dict))

In [ ]:
#deuxième classification : on fait la même chose mais on accorde davantage de poids aux mots rares
#car ils sont plus discriminants, en utilisant TF-IDF 

#étape 1 : calcul de la fréquence de chaque mot 

liste_valeurs = list(keywords_dict.values())
liste_valeurs = [mot for sous_liste in liste_valeurs for mot in sous_liste]
# On crée une liste avec tous nos mots clefs 
liste1 = []
# On remplit une liste vide par les fréquences des mots clefs associés à liste_valeurs
for mot in liste_valeurs:
    i = 0
    for k in df["offre_emploi"]:
        i += len(re.findall(r'\b' + re.escape(mot) + r'\b', k))
    liste1+= [i]
a=sum(liste1)
liste2 = [x / a for x in liste1]
print(liste2)
print(liste_valeurs)

#étape 2 : calcul de la fréquence inverse de chaque terme (mesure de l'importance du terme dans l'ensemble du corpus)

N = len(df)
idf_dict = {}
for mot in liste_valeurs:
    DF = 0
    for k in df["offre_emploi"]:
        # Vérifier que k est une chaîne et que le mot apparaît dans l'offre
        if isinstance(k, str) and re.search(r'\b' + re.escape(mot) + r'\b', k):
            DF += 1
    idf_dict[mot] = math.log(N / (1 + DF))

def classify_offer2(offre): 
    scores = {category: 0 for category in keywords_dict}
    
    for category, keywords in keywords_dict.items():
        scores[category] += sum(offre.count(keyword)*idf_dict.get(keyword, 0) for keyword in keywords)
    
    return max(scores, key=scores.get) if max(scores.values()) > 0 else "inconnu"


df["valeur_dominante_2"] = df["offre_emploi"].apply(lambda x: classify_offer2(x))